In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-03-26


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
last_games

,date,visitor,local,v_line,l_line
0,2024-03-25,CHO,CLE,6.000000,1.142857
1,2024-03-25,BOS,ATL,1.148148,5.900000
2,2024-03-25,DET,NYK,10.500000,1.060606
3,2024-03-25,BRK,TOR,1.408163,3.000000
4,2024-03-25,PHO,SAS,1.125000,6.500000
5,2024-03-25,POR,HOU,6.250000,1.133333
6,2024-03-25,WAS,CHI,7.500000,1.100000
7,2024-03-25,MEM,DEN,9.500000,1.068966
8,2024-03-25,DAL,UTA,1.289855,3.750000
9,2024-03-25,PHI,SAC,4.100000,1.253165


In [6]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

results = [0,0,0,0,1,0,1,1,1]
last_games['Results'] = np.array(results)

In [7]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [8]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [9]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

In [10]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-03-25,CHO,CLE,6.000000,1.142857,0
1,2024-03-25,BOS,ATL,1.148148,5.900000,0
2,2024-03-25,DET,NYK,10.500000,1.060606,0
3,2024-03-25,BRK,TOR,1.408163,3.000000,1
4,2024-03-25,PHO,SAS,1.125000,6.500000,0
5,2024-03-25,POR,HOU,6.250000,1.133333,0
6,2024-03-25,WAS,CHI,7.500000,1.100000,1
7,2024-03-25,MEM,DEN,9.500000,1.068966,0
8,2024-03-25,DAL,UTA,1.289855,3.750000,1
9,2024-03-25,PHI,SAC,4.100000,1.253165,0


In [11]:
prev_games.tail()

,date,visitor,local,v_line,l_line,Results
288,2024-03-24,PHI,LAC,4.50,1.219780,1
289,2024-03-24,CLE,MIA,2.45,1.571429,0
290,2024-03-24,OKC,MIL,2.24,1.675676,0
291,2024-03-24,GSW,MIN,2.20,1.704225,0
292,2024-03-24,IND,LAL,2.30,1.645161,0


In [12]:
prev_games = pd.concat([prev_games, last_games])

In [13]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [14]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [15]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [16]:
tables = soup.findAll('tbody', {'class': 'sportsbook-table__body'})
rows = []
for table in tables:
    for row in table.findAll('tr'):
        rows.append(row)
        
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK').replace('CHA', 'CHO')
    
    try:
        odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')

        if idx == 0:
            game['visitor'] = team_id
            game['v_line'] = american_to_decimal(int(odd))
            idx = 1

        elif idx == 1:
            game['local'] = team_id
            game['l_line'] = american_to_decimal(int(odd))
            idx = 0

            games.append(game)
            game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    except:
        print(f'Error Scraping {team_id} Odds')
        pass
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-03-26,LAL,MIL,4.200000,1.243902
1,2024-03-26,GSW,MIA,1.819672,2.020000
2,2024-03-26,OKC,NOP,1.800000,2.050000
3,2024-03-26,DAL,SAC,1.980392,1.847458


In [17]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)